In [1]:
#import library
from pandas_gbq import read_gbq
import pandas as pd
import numpy as np
import os
import datetime
import ssl
import logging

In [2]:
from datetime import date, timedelta

# Get today's date
today = date.today()
idx = (today.weekday() + 1) % 7

# Current week (CY) range
end_date = today - timedelta(days=(7 + idx - 6))
start_date = end_date - timedelta(days=6)

# Prior period (PP) range
pp_end_date = start_date - timedelta(days=1)
pp_start_date = pp_end_date - timedelta(days=13)

# Corresponding week last year (CWLY and PWLY)
cwly_date = pp_end_date - timedelta(days=364) + timedelta(days=7)
pwly_date = pp_end_date - timedelta(days=364)

# Output all dates
(end_date, start_date, pp_end_date, pp_start_date, cwly_date, pwly_date)


(datetime.date(2025, 9, 6),
 datetime.date(2025, 8, 31),
 datetime.date(2025, 8, 30),
 datetime.date(2025, 8, 17),
 datetime.date(2024, 9, 7),
 datetime.date(2024, 8, 31))

In [3]:
from custom_query import read_sql_query, sql_files

# Read SQL queries from files
queries = {key: read_sql_query(path) for key, path in sql_files.items()}

# Execute queries if all were successfully reada
if all(queries.values()):
    try:
        df_weekly_note = read_gbq(queries["weekly_note"], project_id='pcln-pl-airanalytics-prod')
        finance_data = read_gbq(queries["finance_data"], project_id='pcln-pl-airanalytics-prod')
        deal_share = read_gbq(queries["deal_share"], project_id='pcln-pl-airanalytics-prod')
        tsa_data = read_gbq(queries["tsa_data"], project_id='pcln-pl-airanalytics-prod')
        direct_parity_data = read_gbq(queries["direct_parity_data"], project_id='pcln-pl-airanalytics-prod')
        meta_parity_data = read_gbq(queries["meta_parity_data_query"], project_id='pcln-pl-airanalytics-prod')
        # bookability_data = read_gbq(queries["bookability_query"], project_id='pcln-pl-airanalytics-prod')
        roi_data = read_gbq(queries["roi_query"], project_id='pcln-pl-airanalytics-prod')
        dau_conversion_data = read_gbq(queries["dau_conversion_query"], project_id='pcln-pl-airanalytics-prod')
        # midt_data = read_gbq(queries["midt_query"], project_id='pcln-pl-airanalytics-prod')
        # sem_data = read_gbq(queries["sem_query"], project_id='pcln-pl-airanalytics-prod')
        print("SQL queries executed successfully.")
    except Exception as e:
        print(f"Failed to execute SQL queries: {e}")

/Users/sye/Library/Python/3.9/lib/python/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
SQL queries executed successfully.


## make a copy of dataset

In [4]:
# make a copy of every data set
df_finance=finance_data.copy()
df_direct_parity=direct_parity_data.copy()
df_meta_parity = meta_parity_data.copy()
df_roi=roi_data.copy()
dau_conversion=dau_conversion_data.copy()
df_deal_share=deal_share.copy()
# df_bookability=bookability_data.copy()
df_weekly=df_weekly_note.copy()
df_weekly.columns = df_weekly.columns.str.lower()
# df_midt=midt_data.copy()
# df_sem=sem_data.copy()


In [5]:

base_dir = '../../data_file/'

# Check if directory exists
if not os.path.exists(base_dir):
    print(f"Error: Directory {base_dir} does not exist!")

# Save files
file_names = {
    'df_finance': df_finance,
    'direct_parity': df_direct_parity,
    'meta_parity': df_meta_parity,
    'dau_conversion': dau_conversion,
    'df_roi': df_roi,
    'deal_share': df_deal_share,
    # 'bookability': df_bookability,
    'weekly': df_weekly
}

# Save with verification
for name, df in file_names.items():
    file_path = f'{base_dir}{name}.csv'
    print(f"Saving: {file_path}")
    # print(df.head())  # Print first few rows to confirm data
    df.to_csv(file_path, index=False)

# Read files
for name in file_names.keys():
    file_path = f'{base_dir}{name}.csv'
    print(f"Reading: {file_path}")

    if os.path.exists(file_path):
        df_read = pd.read_csv(file_path)
        # print(df_read.head())  # Print first few rows to compare
    else:
        print(f"Error: {file_path} not found!")

Saving: ../../data_file/df_finance.csv
Saving: ../../data_file/direct_parity.csv
Saving: ../../data_file/meta_parity.csv
Saving: ../../data_file/dau_conversion.csv
Saving: ../../data_file/df_roi.csv
Saving: ../../data_file/deal_share.csv
Saving: ../../data_file/weekly.csv
Reading: ../../data_file/df_finance.csv
Reading: ../../data_file/direct_parity.csv
Reading: ../../data_file/meta_parity.csv
Reading: ../../data_file/dau_conversion.csv
Reading: ../../data_file/df_roi.csv
Reading: ../../data_file/deal_share.csv
Reading: ../../data_file/weekly.csv


/var/folders/k6/ygnww5yd0rx_9c55x7xb7xs00000gn/T/ipykernel_26312/4119341195.py:32: DtypeWarning: Columns (16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_read = pd.read_csv(file_path)


In [6]:
df_weekly

,wk_starting,wk_ending,us_travel_type,company,brand,offer_type,carrier_detail,carrier,offer_method_code,search_channel,...,yoy_gross_contribution,net_contribution_ly,net_contribution_cy,yoy_net_contribution,gr_contr_fee_ly,gr_contr_fee_cy,yoy_gr_contr_fee,net_contr_fee_ly,net_contr_fee_cy,yoy_net_contr_fee
0,2025-08-24,2025-08-30,Domestic,Agoda,Agoda,Flights Only,Alaska Airlines (AS),Alaska Airlines (AS),Retail (Disclosed),Affiliate,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2025-08-24,2025-08-30,Domestic,Agoda,Agoda,Flights Only,Alaska Airlines (AS),Alaska Airlines (AS),Retail (Disclosed),Affiliate,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2025-08-24,2025-08-30,Domestic,Agoda,Agoda,Flights Only,Alaska Airlines (AS),Alaska Airlines (AS),Retail (Disclosed),Affiliate,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2025-08-24,2025-08-30,Domestic,Agoda,Agoda,Flights Only,Alaska Airlines (AS),Alaska Airlines (AS),Retail (Disclosed),Affiliate,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2025-08-24,2025-08-30,Domestic,Agoda,Agoda,Flights Only,Alaska Airlines (AS),Alaska Airlines (AS),Retail (Disclosed),Affiliate,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53362,2025-08-31,2025-09-06,X,Priceline B2C,Priceline,Packages,WestJet (WS),Other,Retail (Disclosed),SEM_Brand,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53363,2025-08-31,2025-09-06,X,Priceline B2C,Priceline,Packages,WestJet (WS),Other,Retail (Disclosed),SEM_Brand,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53364,2025-08-31,2025-09-06,X,Priceline B2C,Priceline,Packages,WestJet (WS),Other,Retail (Disclosed),SEM_Core,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53365,2025-08-31,2025-09-06,X,Priceline B2C,Priceline,Packages,WestJet (WS),Other,Retail (Disclosed),Shop PPC,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
df_finance

,Period,Period_End,Scenario,Gross_Units,Net_Units,gross_cont_revenue_fee,net_cont_revenue_fee
0,1.Reporting Week,2025-09-06,ACTUALS,2.137430e+05,2.016680e+05,2.201964e+06,2.045330e+06
1,1.Reporting Week,2025-09-06,Z-Plan,2.328302e+05,2.165218e+05,2.133191e+06,2.126610e+06
2,2.Previous Week,2025-08-30,ACTUALS,2.151620e+05,2.019770e+05,2.175382e+06,2.015970e+06
3,2.Previous Week,2025-08-30,Z-Plan,2.271974e+05,2.115317e+05,2.111998e+06,2.108918e+06
4,3.MTD,2025-09-06,ACTUALS,1.910000e+05,1.802180e+05,1.929978e+06,1.790626e+06
5,3.MTD,2025-09-06,Z-Plan,2.073748e+05,1.928037e+05,1.911654e+06,1.904550e+06
6,4.QTD,2025-09-06,ACTUALS,2.115913e+06,1.980995e+06,2.122176e+07,1.952466e+07
7,4.QTD,2025-09-06,Z-Plan,2.329378e+06,2.165396e+06,2.070739e+07,2.069428e+07
8,5.YTD,2025-09-06,ACTUALS,7.616390e+06,7.109570e+06,8.034584e+07,7.373846e+07
9,5.YTD,2025-09-06,Z-Plan,8.465398e+06,7.862551e+06,7.318846e+07,7.331505e+07


In [8]:
df_finance.rename(columns={'gross_cont_revenue_fee': 'gross_cont_revenue', 'net_cont_revenue_fee': 'net_cont_revenue'}, inplace=True)
df_finance
df_updated = df_finance[['Gross_Units', 'Net_Units', 'gross_cont_revenue', 'net_cont_revenue']].astype({
    'Gross_Units': 'int',
    'Net_Units': 'int',
    'gross_cont_revenue': 'int',
    'net_cont_revenue': 'int'
})

# If you need to keep the rest of the DataFrame unchanged, you can join the other columns back
df_finance = df_finance.drop(['Gross_Units', 'Net_Units', 'gross_cont_revenue', 'net_cont_revenue'], axis=1).join(df_updated)
df_finance


,Period,Period_End,Scenario,Gross_Units,Net_Units,gross_cont_revenue,net_cont_revenue
0,1.Reporting Week,2025-09-06,ACTUALS,213743,201668,2201964,2045329
1,1.Reporting Week,2025-09-06,Z-Plan,232830,216521,2133190,2126610
2,2.Previous Week,2025-08-30,ACTUALS,215162,201977,2175381,2015969
3,2.Previous Week,2025-08-30,Z-Plan,227197,211531,2111998,2108917
4,3.MTD,2025-09-06,ACTUALS,191000,180218,1929977,1790626
5,3.MTD,2025-09-06,Z-Plan,207374,192803,1911654,1904549
6,4.QTD,2025-09-06,ACTUALS,2115913,1980995,21221758,19524657
7,4.QTD,2025-09-06,Z-Plan,2329378,2165395,20707391,20694277
8,5.YTD,2025-09-06,ACTUALS,7616390,7109570,80345835,73738462
9,5.YTD,2025-09-06,Z-Plan,8465397,7862550,73188461,73315047


In [9]:

def format_number(num):
    if pd.isna(num):
        return ''
    if abs(num) >= 1e6:
        return f"{num/1e6:.1f}M"
    elif abs(num) >= 1e3:
        return f"{num/1e3:.0f}K"
    elif abs(num) < 1e3:
        return "<1K"
    return f"{num:.0f}"

def format_percentage(num):
    if pd.isna(num):
        return ''
    return f"{num:.1f}%"

def round_to_nearest_10(num):
  return round(num / 10) * 10

df_pricelince=df_weekly[(df_weekly['brand']== 'Priceline')]
df_pricelince_air=df_weekly[(df_weekly['brand']== 'Priceline')&(df_weekly['offer_type']== 'Flights Only')]
df_pricelince_b2c=df_weekly[(df_weekly['brand']== 'Priceline')&(df_weekly['company']== 'Priceline B2C')]
df_pricelince_b2c_standalone=df_weekly[(df_weekly['brand']== 'Priceline')&(df_weekly['company']== 'Priceline B2C')&(df_weekly['offer_type']== 'Flights Only')]


In [10]:

# Calculate the percentages
def calculate_percentage(current, previous):
    return (((current - previous) / previous) * 100).round(1)

# Reporting Week
reporting_week_actuals = df_finance[df_finance['Period'] == '1.Reporting Week'].iloc[0]
reporting_week_plan = df_finance[df_finance['Period'] == '6.Reporting Week LY'].iloc[0]

# Previous Week
previous_week_actuals = df_finance[df_finance['Period'] == '2.Previous Week'].iloc[0]
previous_week_plan = df_finance[df_finance['Period'] == '7.Previous Week LY'].iloc[0]



# Calculate metrics
df_finance_ly = {
    'Metric': ['net_tkts_cy', 'gr_tkts_cy', 'net_contribution_cy'
               , 'gross_contribution_cy'
               ],
    'Actual': [
        reporting_week_actuals['Net_Units'],
        reporting_week_actuals['Gross_Units'],
        reporting_week_actuals['net_cont_revenue'],
        reporting_week_actuals['gross_cont_revenue']
    ],
    'Reporting Week': [
        calculate_percentage(reporting_week_actuals['Net_Units'], reporting_week_plan['Net_Units']),
        calculate_percentage(reporting_week_actuals['Gross_Units'], reporting_week_plan['Gross_Units']),
        calculate_percentage(reporting_week_actuals['net_cont_revenue'], reporting_week_plan['net_cont_revenue']),
        # None
        calculate_percentage(reporting_week_actuals['gross_cont_revenue'], reporting_week_plan['gross_cont_revenue'])
    ],
    'Previous Week': [
        calculate_percentage(previous_week_actuals['Net_Units'], previous_week_plan['Net_Units']),
        calculate_percentage(previous_week_actuals['Gross_Units'], previous_week_plan['Gross_Units']),
        calculate_percentage(previous_week_actuals['net_cont_revenue'], previous_week_plan['net_cont_revenue']),
        # None
        calculate_percentage(previous_week_actuals['gross_cont_revenue'], previous_week_plan['gross_cont_revenue'])
    ]

}

df_finance_ly= pd.DataFrame(df_finance_ly)
df_finance_ly['Actual']=df_finance_ly['Actual'].apply(format_number)
df_finance_ly['Reporting Week']=df_finance_ly['Reporting Week'].apply(format_percentage)
df_finance_ly['Previous Week']=df_finance_ly['Previous Week'].apply(format_percentage)
df_finance_ly


,Metric,Actual,Reporting Week,Previous Week
0,net_tkts_cy,202K,-2.7%,-1.4%
1,gr_tkts_cy,214K,-3.6%,-1.8%
2,net_contribution_cy,2.0M,-13.2%,-18.5%
3,gross_contribution_cy,2.2M,-12.6%,-17.7%


In [11]:

# Calculate the percentages
def calculate_percentage(current, previous):
    return (((current - previous) / previous) * 100).round(1)

# Reporting Week
reporting_week_actuals = df_finance[df_finance['Period'] == '1.Reporting Week'].iloc[0]
reporting_week_plan = df_finance[df_finance['Period'] == '1.Reporting Week'].iloc[1]

# Previous Week
previous_week_actuals = df_finance[df_finance['Period'] == '2.Previous Week'].iloc[0]
previous_week_plan = df_finance[df_finance['Period'] == '2.Previous Week'].iloc[1]

# MTD
mtd_actuals = df_finance[df_finance['Period'] == '3.MTD'].iloc[0]
mtd_plan = df_finance[df_finance['Period'] == '3.MTD'].iloc[1]


qtd_actuals = df_finance[df_finance['Period'] == '4.QTD'].iloc[0]
qtd_plan = df_finance[df_finance['Period'] == '4.QTD'].iloc[1]

ytd_actuals = df_finance[df_finance['Period'] == '5.YTD'].iloc[0]
ytd_plan = df_finance[df_finance['Period'] == '5.YTD'].iloc[1]

# Calculate metrics
finance_plan_data = {
    'Metric': ['net_tkts_cy', 'gr_tkts_cy', 'net_contribution_cy'
            #    , 'gross_contribution_cy'
               ],
    'Reporting Week': [
        calculate_percentage(reporting_week_actuals['Net_Units'], reporting_week_plan['Net_Units']),
        calculate_percentage(reporting_week_actuals['Gross_Units'], reporting_week_plan['Gross_Units']),
        calculate_percentage(reporting_week_actuals['net_cont_revenue'], reporting_week_plan['net_cont_revenue'])
        #,calculate_percentage(reporting_week_actuals['gross_cont_revenue'], reporting_week_plan['gross_cont_revenue'])
    ],
    'Previous Week': [
        calculate_percentage(previous_week_actuals['Net_Units'], previous_week_plan['Net_Units']),
        calculate_percentage(previous_week_actuals['Gross_Units'], previous_week_plan['Gross_Units']),
        calculate_percentage(previous_week_actuals['net_cont_revenue'], previous_week_plan['net_cont_revenue'])
        # ,calculate_percentage(previous_week_actuals['gross_cont_revenue'], previous_week_plan['gross_cont_revenue'])
    ],
    'MTD': [
        calculate_percentage(mtd_actuals['Net_Units'], mtd_plan['Net_Units']),
        calculate_percentage(mtd_actuals['Gross_Units'], mtd_plan['Gross_Units']),
        calculate_percentage(mtd_actuals['net_cont_revenue'], mtd_plan['net_cont_revenue'])
        #,calculate_percentage(mtd_actuals['gross_cont_revenue'], mtd_plan['gross_cont_revenue'])
    ],
    'QTD': [
        calculate_percentage(qtd_actuals['Net_Units'], qtd_plan['Net_Units']),
        calculate_percentage(qtd_actuals['Gross_Units'], qtd_plan['Gross_Units']),
        calculate_percentage(qtd_actuals['net_cont_revenue'], qtd_plan['net_cont_revenue'])
        #,calculate_percentage(qtd_actuals['gross_cont_revenue'], qtd_plan['gross_cont_revenue'])
    ],
    'YTD': [
        calculate_percentage(ytd_actuals['Net_Units'], ytd_plan['Net_Units']),
        calculate_percentage(ytd_actuals['Gross_Units'], ytd_plan['Gross_Units']),
        calculate_percentage(ytd_actuals['net_cont_revenue'], ytd_plan['net_cont_revenue'])
        # , calculate_percentage(ytd_actuals['gross_cont_revenue'], ytd_plan['gross_cont_revenue'])
    ]
}

df_finance_plan = pd.DataFrame(finance_plan_data)
df_finance_plan['Reporting Week']=df_finance_plan['Reporting Week'].apply(format_percentage)
df_finance_plan['Previous Week']=df_finance_plan['Previous Week'].apply(format_percentage)
df_finance_plan['MTD']=df_finance_plan['MTD'].apply(format_percentage)
df_finance_plan['QTD']=df_finance_plan['QTD'].apply(format_percentage)
df_finance_plan['YTD']=df_finance_plan['YTD'].apply(format_percentage)
df_finance_plan


,Metric,Reporting Week,Previous Week,MTD,QTD,YTD
0,net_tkts_cy,-6.9%,-4.5%,-6.5%,-8.5%,-9.6%
1,gr_tkts_cy,-8.2%,-5.3%,-7.9%,-9.2%,-10.0%
2,net_contribution_cy,-3.8%,-4.4%,-6.0%,-5.7%,0.6%


In [12]:
df_finance_result= pd.merge(df_finance_ly, df_finance_plan,how='left', on='Metric',suffixes=('','pl'))
df_finance_result = df_finance_result.rename(columns={
        'Reporting Week_ly': 'Reporting Week',
        'Previous Week_ly': 'Previous Week'
        })
df_finance_result=df_finance_result.fillna('')
df_finance_result

,Metric,Actual,Reporting Week,Previous Week,Reporting Weekpl,Previous Weekpl,MTD,QTD,YTD
0,net_tkts_cy,202K,-2.7%,-1.4%,-6.9%,-4.5%,-6.5%,-8.5%,-9.6%
1,gr_tkts_cy,214K,-3.6%,-1.8%,-8.2%,-5.3%,-7.9%,-9.2%,-10.0%
2,net_contribution_cy,2.0M,-13.2%,-18.5%,-3.8%,-4.4%,-6.0%,-5.7%,0.6%
3,gross_contribution_cy,2.2M,-12.6%,-17.7%,,,,,


# TSA

In [13]:
from tsa_table import create_tsa_web_table
df_tsa_web=create_tsa_web_table()
df_tsa_web=df_tsa_web.dropna()

In [14]:
# import tsa_table
# import importlib

# importlib.reload(tsa_table)
# from tsa_table import create_df_tsa_table

# df_tsa=create_df_tsa_table(tsa_data,df_tsa_web,format_percentage,end_date,pp_end_date,cwly_date,pwly_date)
# df_tsa


In [15]:

import subsummary_table as subsummary_table
import importlib

importlib.reload(subsummary_table)
from subsummary_table import create_subsummary_table

df_subsummary = create_subsummary_table(df_pricelince,df_pricelince_air,format_number)
df_normalized=df_subsummary.tail(2)
# .reset_index(names=['Metric'])
df_normalized



,Actual,Reporting Week,Previous Week
normalized_net_tickets_cy,171K,-3.8%,-3.0%
normalized_gross_tickets_cy,181K,-4.8%,-3.6%


In [16]:
df_normalized.loc['DAU','Actual']=0
df_normalized.loc['DAU','Actual']=0
df_normalized.loc['DAU','Reporting Week']=0
df_normalized.loc['DAU','Previous Week']=0

# df_normalized.loc['TSA','Actual']=df_tsa.loc['CY','%']
# df_normalized.loc['TSA','Actual']=df_tsa.loc['CY','%']
# df_normalized.loc['TSA','Reporting Week']=format_percentage(df_tsa.loc['CY','YoY'])
# df_normalized.loc['TSA','Previous Week']=format_percentage(df_tsa.loc['CY_PW','YoY'])

df_normalized.loc['TSA','Actual']=0
df_normalized.loc['TSA','Actual']=0
df_normalized.loc['TSA','Reporting Week']=0
df_normalized.loc['TSA','Previous Week']=0


df_normalized=df_normalized.reset_index(names=['Metric'])
df_normalized

/var/folders/k6/ygnww5yd0rx_9c55x7xb7xs00000gn/T/ipykernel_26312/752893287.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_normalized.loc['DAU','Actual']=0


,Metric,Actual,Reporting Week,Previous Week
0,normalized_net_tickets_cy,171K,-3.8%,-3.0%
1,normalized_gross_tickets_cy,181K,-4.8%,-3.6%
2,DAU,0,0,0
3,TSA,0,0,0


In [17]:
df_summary=pd.concat([df_finance_result, df_normalized]).fillna('').set_index('Metric')
df_summary


,Actual,Reporting Week,Previous Week,Reporting Weekpl,Previous Weekpl,MTD,QTD,YTD
Metric,,,,,,,,
net_tkts_cy,202K,-2.7%,-1.4%,-6.9%,-4.5%,-6.5%,-8.5%,-9.6%
gr_tkts_cy,214K,-3.6%,-1.8%,-8.2%,-5.3%,-7.9%,-9.2%,-10.0%
net_contribution_cy,2.0M,-13.2%,-18.5%,-3.8%,-4.4%,-6.0%,-5.7%,0.6%
gross_contribution_cy,2.2M,-12.6%,-17.7%,,,,,
normalized_net_tickets_cy,171K,-3.8%,-3.0%,,,,,
normalized_gross_tickets_cy,181K,-4.8%,-3.6%,,,,,
DAU,0,0,0,,,,,
TSA,0,0,0,,,,,


In [18]:
import kpi
import importlib

importlib.reload(kpi)

from kpi import calculate_business_metrics

# Calculate business metrics
df_business = calculate_business_metrics(df_pricelince,end_date, pp_end_date)

from kpi import calculate_carrier_metrics
# Calculate carrier metrics
df_carrier = calculate_carrier_metrics(df_pricelince_b2c_standalone, end_date, pp_end_date)

from kpi import calculate_channel_metrics
# Calculate channel metrics
df_channel = calculate_channel_metrics(df_pricelince_b2c_standalone, end_date, pp_end_date)

from kpi import calculate_source_metrics
# Calculate source metrics
df_source = calculate_source_metrics(df_pricelince_b2c_standalone, end_date, pp_end_date)

from kpi import calculate_brand_metrics
# Calculate brand metrics
df_brand = calculate_brand_metrics(df_weekly, end_date, pp_end_date)


In [19]:
df_business

,Net Tickets_standalone,YoY_standalone,YoY PW_standalone,Net Tickets_package,YoY_package,YoY PW_package,Net Tickets_total,YoY_total,YoY PW_total
company,,,,,,,,,
B2C,157K,-8.0%,-6.2%,34K,35.2%,35.6%,191K,-2.5%,-0.9%
B2B,5K,-22.2%,-20.7%,6K,17.1%,4.3%,11K,-5.7%,-9.8%
Total,162K,-8.6%,-6.7%,39K,32.2%,30.3%,202K,-2.7%,-1.4%


In [20]:
df_channel


,Net Tickets_App,YoY_App,YoY PW_App,Net Tickets_Desk/MWEB,YoY_Desk/MWEB,YoY PW_Desk/MWEB,Net Tickets_Total,YoY_Total,YoY PW_Total
search_channel_group,,,,,,,,,
Direct,33K,-2.4%,-4.0%,24K,-22.2%,-22.0%,57K,-11.8%,-12.3%
Web Marketing,8K,76.2%,81.4%,54K,-13.5%,-9.0%,62K,-7.3%,-2.6%
Shop PPC,7K,-1.9%,0.6%,21K,2.3%,4.8%,28K,1.2%,3.7%
Affiliate,<1K,129.3%,110.4%,11K,-14.1%,-15.7%,11K,-13.0%,-14.7%
Total,48K,6.0%,5.3%,109K,-13.1%,-10.5%,157K,-8.0%,-6.2%


In [21]:
df_carrier

,Net Tickets_Retail,YoY_Retail,YoY PW_Retail,Net Tickets_Opaque,YoY_Opaque,YoY PW_Opaque,Net Tickets_Total,YoY_Total,YoY PW_Total
carrier,,,,,,,,,
American Airlines (AA),26K,-28.0%,-26.5%,12K,48.9%,49.3%,38K,-13.9%,-12.0%
Delta Air Lines (DL),19K,-27.1%,-24.2%,<1K,-86.4%,-85.2%,20K,-36.6%,-35.5%
United Airlines (UA),15K,-13.3%,-13.8%,7K,-19.7%,-20.5%,21K,-15.4%,-16.0%
Southwest Airlines (WN),15K,25343.1%,22645.5%,,,,15K,25343.1%,22645.5%
Spirit Airlines (NK),13K,-29.9%,-32.1%,,,,13K,-29.9%,-32.1%
Frontier Airlines (F9),19K,12.5%,29.5%,,,,19K,12.5%,29.5%
Alaska Airlines (AS),4K,-30.3%,-34.0%,3K,-6.3%,-19.4%,7K,-21.2%,-28.4%
JetBlue Airways (B6),7K,10.3%,-2.5%,<1K,inf%,inf%,7K,11.2%,-2.1%
Other,18K,-13.7%,-10.2%,<1K,420.0%,247.7%,18K,-13.1%,-9.7%


In [22]:
df_channel

,Net Tickets_App,YoY_App,YoY PW_App,Net Tickets_Desk/MWEB,YoY_Desk/MWEB,YoY PW_Desk/MWEB,Net Tickets_Total,YoY_Total,YoY PW_Total
search_channel_group,,,,,,,,,
Direct,33K,-2.4%,-4.0%,24K,-22.2%,-22.0%,57K,-11.8%,-12.3%
Web Marketing,8K,76.2%,81.4%,54K,-13.5%,-9.0%,62K,-7.3%,-2.6%
Shop PPC,7K,-1.9%,0.6%,21K,2.3%,4.8%,28K,1.2%,3.7%
Affiliate,<1K,129.3%,110.4%,11K,-14.1%,-15.7%,11K,-13.0%,-14.7%
Total,48K,6.0%,5.3%,109K,-13.1%,-10.5%,157K,-8.0%,-6.2%


In [23]:
df_source

,Net Tickets_Published,YoY_Published,YoY PW_Published,Net Tickets_Private,YoY_Private,YoY PW_Private,Net Tickets_Total,YoY_Total,YoY PW_Total
gds_booking_category,,,,,,,,,
Direct Connect,70K,-1.3%,3.3%,13K,272.5%,266.2%,83K,11.4%,15.7%
Indirect Connect,59K,-19.2%,-16.6%,15K,-35.3%,-41.5%,73K,-23.1%,-23.0%
Phone Sales,,,,1K,-12.9%,-26.0%,1K,-12.9%,-26.0%
Total,128K,-10.4%,-6.7%,29K,4.3%,-3.9%,157K,-8.0%,-6.2%


# MOR


In [24]:
# Create df_mor
df_mor= pd.DataFrame()
df_mor['Net Tickets'] = df_pricelince[(df_pricelince['wk_ending'] == end_date)& ((df_pricelince['merchant_of_record'] == 'VCC')|(df_pricelince['merchant_of_record'] == 'PCLN'))].groupby(['offer_method_code'])['net_tkts_cy'].sum().round(-1).astype(int)
df_mor['Net Tickets_cwly'] = df_pricelince[(df_pricelince['wk_ending'] == end_date)& ((df_pricelince['merchant_of_record'] == 'VCC')|(df_pricelince['merchant_of_record'] == 'PCLN'))].groupby(['offer_method_code'])['net_tkts_ly'].sum().round(-1).astype(int)
df_mor['Net Tickets_PW'] = df_pricelince[(df_pricelince['wk_ending'] == pp_end_date)& ((df_pricelince['merchant_of_record'] == 'VCC')|(df_pricelince['merchant_of_record'] == 'PCLN'))].groupby(['offer_method_code'])['net_tkts_cy'].sum().round(-1).astype(int)
df_mor['Net Tickets_PWly'] = df_pricelince[(df_pricelince['wk_ending'] == pp_end_date)& ((df_pricelince['merchant_of_record'] == 'VCC')|(df_pricelince['merchant_of_record'] == 'PCLN'))].groupby(['offer_method_code'])['net_tkts_ly'].sum().round(-1).astype(int)


# Calculate YoY and YoY PW
df_mor.loc['Total','Net Tickets':'Net Tickets_PWly']=[df_mor['Net Tickets'].sum(),df_mor['Net Tickets_cwly'].sum(),df_mor['Net Tickets_PW'].sum(), df_mor['Net Tickets_PWly'].sum()]


# Create df_mor_1
df_mor_1= pd.DataFrame()
df_mor_1['Net Tickets'] = df_pricelince[(df_pricelince['wk_ending'] == end_date)].groupby(['offer_method_code'])['net_tkts_cy'].sum().round(-1).astype(int)
df_mor_1['Net Tickets_cwly'] = df_pricelince[(df_pricelince['wk_ending'] == end_date)].groupby(['offer_method_code'])['net_tkts_ly'].sum().round(-1).astype(int)
df_mor_1['Net Tickets_PW'] = df_pricelince[(df_pricelince['wk_ending'] == pp_end_date)].groupby(['offer_method_code'])['net_tkts_cy'].sum().round(-1).astype(int)
df_mor_1['Net Tickets_PWly'] = df_pricelince[(df_pricelince['wk_ending'] == pp_end_date)].groupby(['offer_method_code'])['net_tkts_ly'].sum().round(-1).astype(int)
# Calculate YoY and YoY PW
df_mor_1.loc['Total','Net Tickets':'Net Tickets_PWly']=[df_mor_1['Net Tickets'].sum(),df_mor_1['Net Tickets_cwly'].sum(),df_mor_1['Net Tickets_PW'].sum(), df_mor_1['Net Tickets_PWly'].sum()]

df_mor = pd.concat([df_mor,df_mor_1.add_suffix('_total')], axis=1)


df_mor['Actual']=(df_mor['Net Tickets']/df_mor['Net Tickets_total']*100).round(1).apply(format_percentage)
df_mor['%mor_ly']=(df_mor['Net Tickets_cwly']/df_mor['Net Tickets_cwly_total']*100).round().apply(format_percentage)
df_mor['YoY_bps']=(((df_mor['Net Tickets']/df_mor['Net Tickets_total']*100)-(df_mor['Net Tickets_cwly']/df_mor['Net Tickets_cwly_total']*100))*100).round()
df_mor['Actual_PW']=(df_mor['Net Tickets_PW']/df_mor['Net Tickets_PW_total']*100).round().apply(format_percentage)
df_mor['YoY_PW_bps']=(((df_mor['Net Tickets_PW']/df_mor['Net Tickets_PW_total']*100)-(df_mor['Net Tickets_PWly']/df_mor['Net Tickets_PWly_total']*100))*100).round()
df_mor['YoY_bps']=round_to_nearest_10(df_mor['YoY_bps'])
df_mor['YoY_PW_bps']=round_to_nearest_10(df_mor['YoY_PW_bps'])
df_mor=df_mor.loc[['Retail (Disclosed)', 'Total'],['Actual','YoY_bps','YoY_PW_bps']]
df_mor


,Actual,YoY_bps,YoY_PW_bps
offer_method_code,,,
Retail (Disclosed),17.4%,1060.0,920.0
Total,26.7%,880.0,700.0


In [25]:
# Create df_mor
df_mor_2= pd.DataFrame()
df_mor_2['Net Tickets'] = df_pricelince[(df_pricelince['wk_ending'] == end_date)].groupby(['us_travel_type'])['net_tkts_cy'].sum().astype(int)
df_mor_2['Net Tickets_cwly'] = df_pricelince[(df_pricelince['wk_ending'] == end_date)].groupby(['us_travel_type'])['net_tkts_ly'].sum().astype(int)
df_mor_2['Net Tickets_PW'] = df_pricelince[(df_pricelince['wk_ending'] == pp_end_date)].groupby(['us_travel_type'])['net_tkts_cy'].sum().astype(int)
df_mor_2['Net Tickets_PWly'] = df_pricelince[(df_pricelince['wk_ending'] == pp_end_date)].groupby(['us_travel_type'])['net_tkts_ly'].sum().astype(int)
df_mor_2.loc['Total','Net Tickets':'Net Tickets_PWly']=[df_mor_2['Net Tickets'].sum(),df_mor_2['Net Tickets_cwly'].sum(),df_mor_2['Net Tickets_PW'].sum(), df_mor_2['Net Tickets_PWly'].sum()]
actual_cy=(df_mor_2.loc['US Outbound','Net Tickets']*100/df_mor_2.loc['Total','Net Tickets']).round()
actual_ly=df_mor_2.loc['US Outbound','Net Tickets_cwly']*100/df_mor_2.loc['Total','Net Tickets_cwly']
actual_PW=df_mor_2.loc['US Outbound','Net Tickets_PW']*100/df_mor_2.loc['Total','Net Tickets_PW']
actual_PWly=df_mor_2.loc['US Outbound','Net Tickets_PWly']*100/df_mor_2.loc['Total','Net Tickets_PWly']
df_mor.loc['US Outbound','Actual']=format_percentage(actual_cy)
df_mor.loc['US Outbound','YoY_bps']=round_to_nearest_10((actual_cy-actual_ly)*100)
df_mor.loc['US Outbound','YoY_PW_bps']=round_to_nearest_10((actual_PW-actual_PWly)*100)
new_index_order = ['US Outbound','Retail (Disclosed)','Total']
df_mor=df_mor.reindex(new_index_order).fillna('')
df_mor

,Actual,YoY_bps,YoY_PW_bps
offer_method_code,,,
US Outbound,8.0%,-90.0,-80.0
Retail (Disclosed),17.4%,1060.0,920.0
Total,26.7%,880.0,700.0


# DAU


In [26]:
dau_conversion

,actual_date,week_ending,channel,engaged_DAU,converted,conversion
0,2024-05-01,2024-05-04,Direct,144227,9204,0.063816
1,2024-05-01,2024-05-04,Shop PPC Kayak,7293,330,0.045249
2,2024-05-01,2024-05-04,Shop PPC Skyscanner,286,50,0.174825
3,2024-05-01,2024-05-04,Shop PPC Others,15993,465,0.029075
4,2024-05-01,2024-05-04,SEM Brand,25793,1931,0.074865
...,...,...,...,...,...,...
4452,2025-09-07,2025-09-13,Shop PPC Kayak,18932,386,0.020389
4453,2025-09-07,2025-09-13,Shop PPC Others,14970,562,0.037542
4454,2025-09-07,2025-09-13,SEM Core,59893,2244,0.037467
4455,2025-09-08,2025-09-13,Direct,69816,3879,0.055560


In [27]:
df_roi

,week_ending,channel_group,channel,contribution,cost,roi
0,2024-01-06,SEM,SEM Core,712666.550000,745402.240000,0.956083
1,2024-01-06,Shop PPC,Criteo,5942.045950,4871.651700,1.219719
2,2024-01-06,Shop PPC,CHEAPFLIGHTS,196020.059530,158197.808000,1.239082
3,2024-01-06,Shop PPC,GDN,7623.945046,8413.930000,0.906110
4,2024-01-06,SEM,SEM Brand,350375.715323,57057.418927,6.140756
...,...,...,...,...,...,...
812,2026-08-29,Shop PPC,Criteo,0.000000,0.000000,NaN
813,2026-09-05,Shop PPC,GDN,0.000000,0.000000,NaN
814,2026-09-05,Shop PPC,Criteo,0.000000,0.000000,NaN
815,2026-09-12,Shop PPC,Criteo,0.000000,0.000000,NaN


In [28]:

dau_conversion['week_ending'] = pd.to_datetime(dau_conversion['week_ending'], errors='coerce')
df_dau_converison= pd.DataFrame()
df_dau_converison['DAU'] =  dau_conversion[dau_conversion['week_ending'] == end_date.strftime("%Y-%m-%d")].groupby(['channel'])['engaged_DAU'].sum().astype(int)
df_dau_converison['DAU_cwly'] = dau_conversion[dau_conversion['week_ending'] == cwly_date.strftime("%Y-%m-%d")].groupby(['channel'])['engaged_DAU'].sum().astype(int)
df_dau_converison['DAU_pw'] = dau_conversion[dau_conversion['week_ending'] == pp_end_date.strftime("%Y-%m-%d")].groupby(['channel'])['engaged_DAU'].sum().astype(int)
df_dau_converison['DAU_pwly'] = dau_conversion[dau_conversion['week_ending'] == pwly_date.strftime("%Y-%m-%d")].groupby(['channel'])['engaged_DAU'].sum().astype(int)
df_dau_converison['YoY']=(((df_dau_converison['DAU']/df_dau_converison['DAU_cwly'])- 1) * 100).round(1).astype(str) + '%'
df_dau_converison['YoY_PW']=(((df_dau_converison['DAU_pw']/df_dau_converison['DAU_pwly'])-1)* 100).round(1).astype(str) + '%'

df_dau_converison['converted'] =  dau_conversion[dau_conversion['week_ending'] == end_date.strftime("%Y-%m-%d")].groupby(['channel'])['converted'].sum()
df_dau_converison['converted_cwly'] = dau_conversion[dau_conversion['week_ending'] == cwly_date.strftime("%Y-%m-%d")].groupby(['channel'])['converted'].sum()
df_dau_converison['converted_pw'] = dau_conversion[dau_conversion['week_ending'] == pp_end_date.strftime("%Y-%m-%d")].groupby(['channel'])['converted'].sum()
df_dau_converison['converted_pwly'] = dau_conversion[dau_conversion['week_ending'] == pwly_date.strftime("%Y-%m-%d")].groupby(['channel'])['converted'].sum()
# df_dau_converison['converted_YoY']=(((df_dau_converison['Conversion']/df_dau_converison['Conversion_cwly'])-1)* 100).round(1).astype(str) + '%'
# df_dau_converison['converted_YoY_PW']=(((df_dau_converison['Conversion_pw']/df_dau_converison['Conversion_pwly'])-1)* 100).round(1).astype(str) + '%'



df_dau_converison['conversion'] =  ((df_dau_converison['converted']/df_dau_converison['DAU']))
# .round(1).astype(str) + '%'

# Conversion rates for historical dates
df_dau_converison['conversion_cwly'] = df_dau_converison['converted_cwly'] / df_dau_converison['DAU_cwly']
df_dau_converison['conversion_pw'] = df_dau_converison['converted_pw'] / df_dau_converison['DAU_pw']
df_dau_converison['conversion_pwly'] = df_dau_converison['converted_pwly'] / df_dau_converison['DAU_pwly']

# YoY and YoY_PW for conversion
df_dau_converison['conversion_YoY'] = (
    ((df_dau_converison['conversion'] / df_dau_converison['conversion_cwly']) - 1) * 100
).round(1).astype(str) + '%'

df_dau_converison['conversion_YoY_PW'] = (
    ((df_dau_converison['conversion_pw'] / df_dau_converison['conversion_pwly']) - 1) * 100
).round(1).astype(str) + '%'

# df_dau_converison=df_dau_converison.reset_index(channel)
df_dau_converison


,DAU,DAU_cwly,DAU_pw,DAU_pwly,YoY,YoY_PW,converted,converted_cwly,converted_pw,converted_pwly,conversion,conversion_cwly,conversion_pw,conversion_pwly,conversion_YoY,conversion_YoY_PW
channel,,,,,,,,,,,,,,,,
Affiliate,11159,7356,11702,7119,51.7%,64.4%,1929,1318,2093,1275,0.172865,0.179173,0.178858,0.179098,-3.5%,-0.1%
Direct,798044,779445,806537,855340,2.4%,-5.7%,46897,49518,46556,52952,0.058765,0.06353,0.057723,0.061908,-7.5%,-6.8%
SEM Brand,137826,146610,140928,150168,-6.0%,-6.2%,9617,10766,9708,10559,0.069776,0.073433,0.068886,0.070315,-5.0%,-2.0%
SEM Core,429264,456317,464002,452274,-5.9%,2.6%,18266,20641,18673,20042,0.042552,0.045234,0.040243,0.044314,-5.9%,-9.2%
Shop PPC Cheapflights,245297,245136,256726,240923,0.1%,6.6%,3420,4256,3522,4218,0.013942,0.017362,0.013719,0.017508,-19.7%,-21.6%
Shop PPC Google,10842,23186,11467,20780,-53.2%,-44.8%,1428,756,1556,826,0.13171,0.032606,0.135694,0.03975,303.9%,241.4%
Shop PPC Kayak,168819,42006,176830,40404,301.9%,337.7%,2686,1724,2807,1680,0.015911,0.041042,0.015874,0.04158,-61.2%,-61.8%
Shop PPC Others,112902,104514,122861,107119,8.0%,14.7%,4542,4531,4068,4634,0.04023,0.043353,0.033111,0.04326,-7.2%,-23.5%
Shop PPC Skyscanner,1675,1188,1700,1272,41.0%,33.6%,354,244,388,237,0.211343,0.205387,0.228235,0.186321,2.9%,22.5%


In [29]:
df_dau_converison['conversion']=df_dau_converison['conversion'].round(1).astype(str) + '%'
df_dau_converison

,DAU,DAU_cwly,DAU_pw,DAU_pwly,YoY,YoY_PW,converted,converted_cwly,converted_pw,converted_pwly,conversion,conversion_cwly,conversion_pw,conversion_pwly,conversion_YoY,conversion_YoY_PW
channel,,,,,,,,,,,,,,,,
Affiliate,11159,7356,11702,7119,51.7%,64.4%,1929,1318,2093,1275,0.2%,0.179173,0.178858,0.179098,-3.5%,-0.1%
Direct,798044,779445,806537,855340,2.4%,-5.7%,46897,49518,46556,52952,0.1%,0.06353,0.057723,0.061908,-7.5%,-6.8%
SEM Brand,137826,146610,140928,150168,-6.0%,-6.2%,9617,10766,9708,10559,0.1%,0.073433,0.068886,0.070315,-5.0%,-2.0%
SEM Core,429264,456317,464002,452274,-5.9%,2.6%,18266,20641,18673,20042,0.0%,0.045234,0.040243,0.044314,-5.9%,-9.2%
Shop PPC Cheapflights,245297,245136,256726,240923,0.1%,6.6%,3420,4256,3522,4218,0.0%,0.017362,0.013719,0.017508,-19.7%,-21.6%
Shop PPC Google,10842,23186,11467,20780,-53.2%,-44.8%,1428,756,1556,826,0.1%,0.032606,0.135694,0.03975,303.9%,241.4%
Shop PPC Kayak,168819,42006,176830,40404,301.9%,337.7%,2686,1724,2807,1680,0.0%,0.041042,0.015874,0.04158,-61.2%,-61.8%
Shop PPC Others,112902,104514,122861,107119,8.0%,14.7%,4542,4531,4068,4634,0.0%,0.043353,0.033111,0.04326,-7.2%,-23.5%
Shop PPC Skyscanner,1675,1188,1700,1272,41.0%,33.6%,354,244,388,237,0.2%,0.205387,0.228235,0.186321,2.9%,22.5%


In [30]:
df_roi[df_roi['week_ending'] == end_date.strftime("%Y-%m-%d")].groupby(['channel'])['contribution'].sum().astype(int)

Series([], Name: contribution, dtype: int64)

In [31]:

df_roi_v1= pd.DataFrame()
df_roi['week_ending'] = pd.to_datetime(df_roi['week_ending'], errors='coerce')

df_roi_v1['contribution'] =  df_roi[df_roi['week_ending'] == end_date.strftime("%Y-%m-%d")].groupby(['channel'])['contribution'].sum().astype(int)
df_roi_v1['contribution_cwly'] = df_roi[df_roi['week_ending'] == cwly_date.strftime("%Y-%m-%d")].groupby(['channel'])['contribution'].sum().astype(int)
df_roi_v1['contribution_pw'] = df_roi[df_roi['week_ending'] == pp_end_date.strftime("%Y-%m-%d")].groupby(['channel'])['contribution'].sum().astype(int)
df_roi_v1['contribution_pwly'] = df_roi[df_roi['week_ending'] == pwly_date.strftime("%Y-%m-%d")].groupby(['channel'])['contribution'].sum().astype(int)


df_roi_v1['cost'] =  df_roi[df_roi['week_ending'] == end_date.strftime("%Y-%m-%d")].groupby(['channel'])['cost'].sum()
df_roi_v1['cost_cwly'] = df_roi[df_roi['week_ending'] == cwly_date.strftime("%Y-%m-%d")].groupby(['channel'])['cost'].sum()
df_roi_v1['cost_pw'] = df_roi[df_roi['week_ending'] == pp_end_date.strftime("%Y-%m-%d")].groupby(['channel'])['cost'].sum()
df_roi_v1['cost_pwly'] = df_roi[df_roi['week_ending'] == pwly_date.strftime("%Y-%m-%d")].groupby(['channel'])['cost'].sum()


df_roi_v1['ROI'] =  df_roi_v1['contribution']/df_roi_v1['cost']
df_roi_v1['ROI_cwly'] = df_roi_v1['contribution_cwly'] / df_roi_v1['cost_cwly']
df_roi_v1['ROI_pw'] = df_roi_v1['contribution_pw'] / df_roi_v1['cost_pw']
df_roi_v1['ROI_pwly'] = df_roi_v1['contribution_pwly'] / df_roi_v1['cost_pwly']

# YoY and YoY_PW for conversion
df_roi_v1['ROI_YoY'] = (
    ((df_roi_v1['ROI'] / df_roi_v1['ROI_cwly']) - 1) * 100
).round(1).astype(str) + '%'

df_roi_v1['ROI_YoY_PW'] = (
    ((df_roi_v1['ROI_pw'] / df_roi_v1['ROI_pwly']) - 1) * 100
).round(1).astype(str) + '%'

df_roi_v1



,contribution,contribution_cwly,contribution_pw,contribution_pwly,cost,cost_cwly,cost_pw,cost_pwly,ROI,ROI_cwly,ROI_pw,ROI_pwly,ROI_YoY,ROI_YoY_PW
channel,,,,,,,,,,,,,,
CHEAPFLIGHTS,163181,192627,156485,189117,149649.416000,161693.57600,153612.336000,1.633088e+05,1.090422,1.191309,1.018701,1.158033,-8.5%,-12.0%
CJ,41921,139393,40921,138058,7140.725000,57602.65600,7442.231000,5.969129e+04,5.870692,2.419906,5.498486,2.312867,142.6%,137.7%
CLICKTRIPZ,3864,4000,5358,6247,2591.710000,3149.85000,3514.520000,3.148230e+03,1.490908,1.269902,1.524533,1.984290,17.4%,-23.2%
Criteo,5404,6293,5974,6483,4093.842400,8754.37410,3991.800500,6.065961e+03,1.320031,0.718841,1.496568,1.068751,83.6%,40.0%
GDN,22921,9521,26715,10627,16158.840000,8006.96000,16199.820000,8.367870e+03,1.418481,1.189090,1.649092,1.269977,19.3%,29.9%
Kayak,25248,19928,21045,18543,19704.248000,16921.70400,19454.968000,1.628716e+04,1.281348,1.177659,1.081729,1.138504,8.8%,-5.0%
SEM Brand,374011,392389,374143,399775,67334.788061,63599.31982,69151.266968,6.694280e+04,5.554499,6.169704,5.410501,5.971890,-10.0%,-9.4%
SEM Core,920345,925704,900763,919290,892300.410000,992106.09000,926124.610000,1.051823e+06,1.031430,0.933070,0.972615,0.873997,10.5%,11.3%
SKYSCANNER,5737,5428,5287,4792,5071.100000,3672.85000,5133.150000,3.932100e+03,1.131313,1.477871,1.029972,1.218687,-23.4%,-15.5%


In [32]:
df_roi_v1 = df_roi_v1.rename(index={
        'CHEAPFLIGHTS': 'Shop PPC Cheapflights',
        'CJ': 'Affiliate',
        'CLICKTRIPZ':'Shop PPC Others',
        'SKYSCANNER' :'Shop PPC Skyscanner',
        'Kayak': 'Shop PPC Kayak'
        })

In [33]:
df_roi_v1

,contribution,contribution_cwly,contribution_pw,contribution_pwly,cost,cost_cwly,cost_pw,cost_pwly,ROI,ROI_cwly,ROI_pw,ROI_pwly,ROI_YoY,ROI_YoY_PW
channel,,,,,,,,,,,,,,
Shop PPC Cheapflights,163181,192627,156485,189117,149649.416000,161693.57600,153612.336000,1.633088e+05,1.090422,1.191309,1.018701,1.158033,-8.5%,-12.0%
Affiliate,41921,139393,40921,138058,7140.725000,57602.65600,7442.231000,5.969129e+04,5.870692,2.419906,5.498486,2.312867,142.6%,137.7%
Shop PPC Others,3864,4000,5358,6247,2591.710000,3149.85000,3514.520000,3.148230e+03,1.490908,1.269902,1.524533,1.984290,17.4%,-23.2%
Criteo,5404,6293,5974,6483,4093.842400,8754.37410,3991.800500,6.065961e+03,1.320031,0.718841,1.496568,1.068751,83.6%,40.0%
GDN,22921,9521,26715,10627,16158.840000,8006.96000,16199.820000,8.367870e+03,1.418481,1.189090,1.649092,1.269977,19.3%,29.9%
Shop PPC Kayak,25248,19928,21045,18543,19704.248000,16921.70400,19454.968000,1.628716e+04,1.281348,1.177659,1.081729,1.138504,8.8%,-5.0%
SEM Brand,374011,392389,374143,399775,67334.788061,63599.31982,69151.266968,6.694280e+04,5.554499,6.169704,5.410501,5.971890,-10.0%,-9.4%
SEM Core,920345,925704,900763,919290,892300.410000,992106.09000,926124.610000,1.051823e+06,1.031430,0.933070,0.972615,0.873997,10.5%,11.3%
Shop PPC Skyscanner,5737,5428,5287,4792,5071.100000,3672.85000,5133.150000,3.932100e+03,1.131313,1.477871,1.029972,1.218687,-23.4%,-15.5%


In [34]:
df_dau_converison.reset_index()

,channel,DAU,DAU_cwly,DAU_pw,DAU_pwly,YoY,YoY_PW,converted,converted_cwly,converted_pw,converted_pwly,conversion,conversion_cwly,conversion_pw,conversion_pwly,conversion_YoY,conversion_YoY_PW
0,Affiliate,11159,7356,11702,7119,51.7%,64.4%,1929,1318,2093,1275,0.2%,0.179173,0.178858,0.179098,-3.5%,-0.1%
1,Direct,798044,779445,806537,855340,2.4%,-5.7%,46897,49518,46556,52952,0.1%,0.06353,0.057723,0.061908,-7.5%,-6.8%
2,SEM Brand,137826,146610,140928,150168,-6.0%,-6.2%,9617,10766,9708,10559,0.1%,0.073433,0.068886,0.070315,-5.0%,-2.0%
3,SEM Core,429264,456317,464002,452274,-5.9%,2.6%,18266,20641,18673,20042,0.0%,0.045234,0.040243,0.044314,-5.9%,-9.2%
4,Shop PPC Cheapflights,245297,245136,256726,240923,0.1%,6.6%,3420,4256,3522,4218,0.0%,0.017362,0.013719,0.017508,-19.7%,-21.6%
5,Shop PPC Google,10842,23186,11467,20780,-53.2%,-44.8%,1428,756,1556,826,0.1%,0.032606,0.135694,0.03975,303.9%,241.4%
6,Shop PPC Kayak,168819,42006,176830,40404,301.9%,337.7%,2686,1724,2807,1680,0.0%,0.041042,0.015874,0.04158,-61.2%,-61.8%
7,Shop PPC Others,112902,104514,122861,107119,8.0%,14.7%,4542,4531,4068,4634,0.0%,0.043353,0.033111,0.04326,-7.2%,-23.5%
8,Shop PPC Skyscanner,1675,1188,1700,1272,41.0%,33.6%,354,244,388,237,0.2%,0.205387,0.228235,0.186321,2.9%,22.5%


In [35]:
df_merged = (
    df_dau_converison.reset_index()   # channel becomes a column
    .merge(
        df_roi_v1.reset_index(),         # channelgroup becomes a column
        left_on="channel",
        right_on="channel",
        how="left"
    )
)
df_merged 

,channel,DAU,DAU_cwly,DAU_pw,DAU_pwly,YoY,YoY_PW,converted,converted_cwly,converted_pw,...,cost,cost_cwly,cost_pw,cost_pwly,ROI,ROI_cwly,ROI_pw,ROI_pwly,ROI_YoY,ROI_YoY_PW
0,Affiliate,11159,7356,11702,7119,51.7%,64.4%,1929,1318,2093,...,7140.725000,57602.65600,7442.231000,5.969129e+04,5.870692,2.419906,5.498486,2.312867,142.6%,137.7%
1,Direct,798044,779445,806537,855340,2.4%,-5.7%,46897,49518,46556,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SEM Brand,137826,146610,140928,150168,-6.0%,-6.2%,9617,10766,9708,...,67334.788061,63599.31982,69151.266968,6.694280e+04,5.554499,6.169704,5.410501,5.971890,-10.0%,-9.4%
3,SEM Core,429264,456317,464002,452274,-5.9%,2.6%,18266,20641,18673,...,892300.410000,992106.09000,926124.610000,1.051823e+06,1.031430,0.933070,0.972615,0.873997,10.5%,11.3%
4,Shop PPC Cheapflights,245297,245136,256726,240923,0.1%,6.6%,3420,4256,3522,...,149649.416000,161693.57600,153612.336000,1.633088e+05,1.090422,1.191309,1.018701,1.158033,-8.5%,-12.0%
5,Shop PPC Google,10842,23186,11467,20780,-53.2%,-44.8%,1428,756,1556,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Shop PPC Kayak,168819,42006,176830,40404,301.9%,337.7%,2686,1724,2807,...,19704.248000,16921.70400,19454.968000,1.628716e+04,1.281348,1.177659,1.081729,1.138504,8.8%,-5.0%
7,Shop PPC Others,112902,104514,122861,107119,8.0%,14.7%,4542,4531,4068,...,2591.710000,3149.85000,3514.520000,3.148230e+03,1.490908,1.269902,1.524533,1.984290,17.4%,-23.2%
8,Shop PPC Skyscanner,1675,1188,1700,1272,41.0%,33.6%,354,244,388,...,5071.100000,3672.85000,5133.150000,3.932100e+03,1.131313,1.477871,1.029972,1.218687,-23.4%,-15.5%


In [36]:
df_merged = df_merged.set_index('channel')
df_merged

,DAU,DAU_cwly,DAU_pw,DAU_pwly,YoY,YoY_PW,converted,converted_cwly,converted_pw,converted_pwly,...,cost,cost_cwly,cost_pw,cost_pwly,ROI,ROI_cwly,ROI_pw,ROI_pwly,ROI_YoY,ROI_YoY_PW
channel,,,,,,,,,,,,,,,,,,,,,
Affiliate,11159,7356,11702,7119,51.7%,64.4%,1929,1318,2093,1275,...,7140.725000,57602.65600,7442.231000,5.969129e+04,5.870692,2.419906,5.498486,2.312867,142.6%,137.7%
Direct,798044,779445,806537,855340,2.4%,-5.7%,46897,49518,46556,52952,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SEM Brand,137826,146610,140928,150168,-6.0%,-6.2%,9617,10766,9708,10559,...,67334.788061,63599.31982,69151.266968,6.694280e+04,5.554499,6.169704,5.410501,5.971890,-10.0%,-9.4%
SEM Core,429264,456317,464002,452274,-5.9%,2.6%,18266,20641,18673,20042,...,892300.410000,992106.09000,926124.610000,1.051823e+06,1.031430,0.933070,0.972615,0.873997,10.5%,11.3%
Shop PPC Cheapflights,245297,245136,256726,240923,0.1%,6.6%,3420,4256,3522,4218,...,149649.416000,161693.57600,153612.336000,1.633088e+05,1.090422,1.191309,1.018701,1.158033,-8.5%,-12.0%
Shop PPC Google,10842,23186,11467,20780,-53.2%,-44.8%,1428,756,1556,826,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Shop PPC Kayak,168819,42006,176830,40404,301.9%,337.7%,2686,1724,2807,1680,...,19704.248000,16921.70400,19454.968000,1.628716e+04,1.281348,1.177659,1.081729,1.138504,8.8%,-5.0%
Shop PPC Others,112902,104514,122861,107119,8.0%,14.7%,4542,4531,4068,4634,...,2591.710000,3149.85000,3514.520000,3.148230e+03,1.490908,1.269902,1.524533,1.984290,17.4%,-23.2%
Shop PPC Skyscanner,1675,1188,1700,1272,41.0%,33.6%,354,244,388,237,...,5071.100000,3672.85000,5133.150000,3.932100e+03,1.131313,1.477871,1.029972,1.218687,-23.4%,-15.5%


In [116]:
df_roi_v=df_merged[['DAU','YoY','YoY_PW','conversion','conversion_YoY','conversion_YoY_PW','ROI','ROI_YoY','ROI_YoY_PW']]
new_cols = [
    "DAU", "YoY", "YoY PW",
    "Conversion", "YoY", "YoY PW",
    "ROI", "YoY", "YoY PW",
]
df_roi_v.columns = new_cols
df_roi_v['DAU'] = df_roi_v['DAU'].apply(format_number)
df_roi_v['ROI'] = df_roi_v['ROI'].round(2)
df_roi_v=df_roi_v.reindex(['Direct', 'SEM Core', 'SEM Brand','Shop PPC Cheapflights', 'Shop PPC Google', 'Shop PPC Kayak', 'Shop PPC Skyscanner', 'Shop PPC Others','Affiliate']).fillna('')
df_roi_v

/var/folders/k6/ygnww5yd0rx_9c55x7xb7xs00000gn/T/ipykernel_26312/184942341.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_roi_v['ROI'] = df_roi_v['ROI'].round(2)


,DAU,YoY,YoY PW,Conversion,YoY,YoY PW,ROI,YoY,YoY PW
channel,,,,,,,,,
Direct,798044,2.4%,-5.7%,0.1%,-7.5%,-6.8%,,,
SEM Core,429264,-5.9%,2.6%,0.0%,-5.9%,-9.2%,1.03,10.5%,11.3%
SEM Brand,137826,-6.0%,-6.2%,0.1%,-5.0%,-2.0%,5.55,-10.0%,-9.4%
Shop PPC Cheapflights,245297,0.1%,6.6%,0.0%,-19.7%,-21.6%,1.09,-8.5%,-12.0%
Shop PPC Google,10842,-53.2%,-44.8%,0.1%,303.9%,241.4%,,,
Shop PPC Kayak,168819,301.9%,337.7%,0.0%,-61.2%,-61.8%,1.28,8.8%,-5.0%
Shop PPC Skyscanner,1675,41.0%,33.6%,0.2%,2.9%,22.5%,1.13,-23.4%,-15.5%
Shop PPC Others,112902,8.0%,14.7%,0.0%,-7.2%,-23.5%,1.49,17.4%,-23.2%
Affiliate,11159,51.7%,64.4%,0.2%,-3.5%,-0.1%,5.87,142.6%,137.7%


In [121]:
format_number(df_roi_v['DAU'].sum())

'1.9M'

## ROI

In [59]:

from dau_roi_table import create_roi_table

import importlib, dau_roi_table
importlib.reload(dau_roi_table)


# 1) Compute raw DAU/Conversion
df_dau_v1= calculate_dau_conversion(
    dau_conversion, end_date, pp_end_date, cwly_date, pwly_date
)

# 2) Compute ROI
df_roi_v1 = calculate_roi(
    df_roi, end_date, pp_end_date, cwly_date, pwly_date
)

# 3) Build the presentation table
df_roi_v = create_roi_table(df_roi_levels, df_dau_conv_raw, end_date, pp_end_date, cwly_date, pwly_date)




NameError: name 'calculate_dau_conversion' is not defined

## Parity data

In [40]:
from parity_table import create_parity_table

df_parity = create_parity_table(
        format_percentage,
        df_direct_parity,
        round_to_nearest_10,
        df_meta_parity)
df_parity

,Actual_pcln,YoY (bps)_pcln,YoY PW (bps)_pcln,Actual_expe,YoY (bps)_expe,YoY PW (bps)_expe
Direct vs Expedia,96.2%,330.0,390.0,NaN,NaN,NaN
Kayak Placement,22.8%,1170.0,990.0,19.4%,920.0,870.0
Skyscanner Placement,18.7%,1450.0,1780.0,30.9%,-1330.0,-1310.0


In [41]:
df_deal_share

,deal_vs_non_deal,yr_mnth,wk_ending,offer_type,fare_type,totalTkts
0,Deal,2023-01,2023-01-07,Air,Web Fare,2790.0
1,Deal,2023-01,2023-01-07,Air,splitTkts,29166.0
2,Deal,2023-01,2023-01-07,Air,Opaque,13650.0
3,Deal,2023-01,2023-01-07,Air,Retail Net,127.0
4,No Deal,2023-01,2023-01-07,Air,Retail Published,95674.0
...,...,...,...,...,...,...
2323,Deal,2025-09,2025-09-13,PKG,Web Fare,559.0
2324,Deal,2025-09,2025-09-13,PKG,Mixed,4.0
2325,Deal,2025-09,2025-09-13,PKG,splitTkts,2644.0
2326,No Deal,2025-09,2025-09-13,PKG,Retail Published,4979.0


In [42]:
deal_ticket_actual_cy=df_deal_share[(df_deal_share['wk_ending']==end_date)].groupby('deal_vs_non_deal')['totalTkts'].sum()
deal_ticket_actual_cy=deal_ticket_actual_cy[0]/(deal_ticket_actual_cy[0]+deal_ticket_actual_cy[1])
# deal_ticket_actual_cy=format_percentage(deal_ticket_actual_cy*100)

deal_ticket_actual_cy

deal_ticket_actual_pw=df_deal_share[(df_deal_share['wk_ending']==pp_end_date)].groupby('deal_vs_non_deal')['totalTkts'].sum()
deal_ticket_actual_pw=deal_ticket_actual_pw[0]/(deal_ticket_actual_pw[0]+deal_ticket_actual_pw[1])
# deal_ticket_actual_pw=format_percentage(deal_ticket_actual_pw*100)
deal_ticket_actual_pw

deal_ticket_actual_cwly=df_deal_share[(df_deal_share['wk_ending']==cwly_date)].groupby('deal_vs_non_deal')['totalTkts'].sum()
deal_ticket_actual_cwly=deal_ticket_actual_cwly[0]/(deal_ticket_actual_cwly[0]+deal_ticket_actual_cwly[1])
# deal_ticket_actual_cwly=format_percentage(deal_ticket_actual_cwly*100)
deal_ticket_actual_cwly

deal_ticket_actual_pwly=df_deal_share[(df_deal_share['wk_ending']==pwly_date)].groupby('deal_vs_non_deal')['totalTkts'].sum()
deal_ticket_actual_pwly=deal_ticket_actual_pwly[0]/(deal_ticket_actual_pwly[0]+deal_ticket_actual_pwly[1])
# deal_ticket_actual_pwly=format_percentage(deal_ticket_actual_pwly*100)
deal_ticket_actual_pwly

/var/folders/k6/ygnww5yd0rx_9c55x7xb7xs00000gn/T/ipykernel_26312/2117931116.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  deal_ticket_actual_cy=deal_ticket_actual_cy[0]/(deal_ticket_actual_cy[0]+deal_ticket_actual_cy[1])
/var/folders/k6/ygnww5yd0rx_9c55x7xb7xs00000gn/T/ipykernel_26312/2117931116.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  deal_ticket_actual_pw=deal_ticket_actual_pw[0]/(deal_ticket_actual_pw[0]+deal_ticket_actual_pw[1])
/var/folders/k6/ygnww5yd0rx_9c55x7xb7xs00000gn/T/ipykernel_26312/2117931116.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future

np.float64(0.3593129533908645)

## MIDT

In [43]:



# df_midt_data= pd.DataFrame()
# #Calulate actual
# a=df_midt[(df_midt['wk_ending']==end_date)].groupby(['agency'])['Tickets'].sum()['PRICELINE']
# b=df_midt[(df_midt['wk_ending']==end_date)]['Tickets'].sum()
# # Calculate the percentage
# Actual = ((a * 100) / b)


# #Calulate cwly_date
# a=df_midt[(df_midt['wk_ending']==cwly_date)].groupby(['agency'])['Tickets'].sum()['PRICELINE']
# b=df_midt[(df_midt['wk_ending']==cwly_date)]['Tickets'].sum()
# # Calculate the percentage
# Actual_cwly = ((a * 100) / b)

# #Calulate PWly_date
# a=df_midt[(df_midt['wk_ending']==pp_end_date)].groupby(['agency'])['Tickets'].sum()['PRICELINE']
# b=df_midt[(df_midt['wk_ending']==pp_end_date)]['Tickets'].sum()
# # Calculate the percentage
# actual_PW = ((a * 100) / b)

# #Calulate PWly_date
# a=df_midt[(df_midt['wk_ending']==PWly_date)].groupby(['agency'])['Tickets'].sum()['PRICELINE']
# b=df_midt[(df_midt['wk_ending']==PWly_date)]['Tickets'].sum()
# # Calculate the percentage
# actual_PWly = ((a * 100) / b)


# # Create a new DataFrame with the calculated 'Actual' column
# df_midt_data = pd.DataFrame()
# df_midt_data['Actual']=[format_percentage(Actual)]
# df_midt_data['YoY (bps)']=[round_to_nearest_10((Actual-Actual_cwly)*100)]
# df_midt_data['YoY PW(bps)']=[round_to_nearest_10((actual_PW-actual_PWly)*100)]
# df_midt_data
# # df_midt_data['Actual']

## SEM

In [44]:
# df_sem=df_sem[(df_sem['DOMAIN']=='You')|(df_sem['DOMAIN']=='expedia.com')]
# df_sem
# # Calculate actuals for current, cwly, pp, and pwly dates
# def compute_rate_ratio(df, date):
#     grouped = df[df['wk_ending'] == date].groupby('DOMAIN')[['ABS_TOP_PAGE_RATE_IMPRESSIONS', 'TOTAL_IMPRESSIONS']].sum()
#     grouped['RATE_RATIO'] = grouped['ABS_TOP_PAGE_RATE_IMPRESSIONS'] / grouped['TOTAL_IMPRESSIONS']
#     return grouped['RATE_RATIO']  # return only the RATE_RATIO series

# # Compute each set
# actual = compute_rate_ratio(df_sem, end_date)
# actual_cwly = compute_rate_ratio(df_sem, cwly_date)
# actual_pw = compute_rate_ratio(df_sem, pp_end_date)
# actual_pwly = compute_rate_ratio(df_sem, pwly_date)

# df_sem_data = pd.DataFrame({
#     'Actual': (actual*100).apply(format_percentage),
#     'YoY (bps)': round_to_nearest_10((actual - actual_cwly) * 10000),
#     'YoY PW(bps)': round_to_nearest_10((actual_pw - actual_pwly) * 10000)
# })

# df_sem_data.reset_index(inplace=True)  # optional, for clarity if you need DOMAIN as a column
# df_sem_data


In [45]:
df_others = pd.DataFrame()
df_others.loc['deal_share','Actual_pcln']=format_percentage(deal_ticket_actual_cy*100)
df_others.loc['deal_share','YoY (bps)_pcln']=round_to_nearest_10((deal_ticket_actual_cy-deal_ticket_actual_cwly)*10000)
df_others.loc['deal_share','YoY PW (bps)_pcln']=round_to_nearest_10((deal_ticket_actual_pw-deal_ticket_actual_pwly)*10000)


# df_others.loc['MIDT','Actual_pcln']=df_midt_data.loc[0,'Actual']
# df_others.loc['MIDT','YoY (bps)_pcln']=df_midt_data.loc[0,'YoY (bps)']
# df_others.loc['MIDT','YoY PW (bps)_pcln']=df_midt_data.loc[0,'YoY PW(bps)']

# df_others.loc['SEM Impressions','Actual_pcln']=df_sem_data.loc[0,'Actual']
# df_others.loc['SEM Impressions','YoY (bps)_pcln']=df_sem_data.loc[0,'YoY (bps)']
# df_others.loc['SEM Impressions','YoY PW (bps)_pcln']=df_sem_data.loc[0,'YoY PW(bps)']

# df_others.loc['SEM Impressions','Actual_expe']=df_sem_data.loc[1,'Actual']
# df_others.loc['SEM Impressions','YoY (bps)_expe']=df_sem_data.loc[1,'YoY (bps)']
# df_others.loc['SEM Impressions','YoY PW (bps)_expe']=df_sem_data.loc[1,'YoY PW(bps)']

# df_others.loc['TSA','Actual_pcln']=df_tsa.loc['CY','%']
# df_others.loc['TSA','YoY (bps)_pcln']=df_tsa.loc['CY','YoY']
# df_others.loc['TSA','YoY PW (bps)_pcln']=df_tsa.loc['CY_PW','YoY']

df_others.loc['US Outbound','Actual_pcln']=df_mor.loc['US Outbound','Actual']
df_others.loc['US Outbound','YoY (bps)_pcln']=df_mor.loc['US Outbound','YoY_bps']
df_others.loc['US Outbound','YoY PW (bps)_pcln']=df_mor.loc['US Outbound','YoY_PW_bps']
df_others.loc['Merchant Retail','Actual_pcln']=df_mor.loc['Retail (Disclosed)','Actual']
df_others.loc['Merchant Retail','YoY (bps)_pcln']=df_mor.loc['Retail (Disclosed)','YoY_bps']
df_others.loc['Merchant Retail','YoY PW (bps)_pcln']=df_mor.loc['Retail (Disclosed)','YoY_PW_bps']
df_others.loc['Merchant Total','Actual_pcln']=df_mor.loc['Total','Actual']
df_others.loc['Merchant Total','YoY (bps)_pcln']=df_mor.loc['Total','YoY_bps']
df_others.loc['Merchant Total','YoY PW (bps)_pcln']=df_mor.loc['Total','YoY_PW_bps']


# df_others = df_others.append(df_parity.iloc[0:])
df_others= pd.concat([df_others, df_parity.iloc[0:]])
new_index_order = [ 'ARC','MIDT','SEM Impressions','Direct vs Expedia','Kayak Placement','Skyscanner Placement', 'US Outbound','Merchant Retail','Merchant Total','deal_share','Bookability']
df_others=df_others.reindex(new_index_order).fillna('')
df_others

,Actual_pcln,YoY (bps)_pcln,YoY PW (bps)_pcln,Actual_expe,YoY (bps)_expe,YoY PW (bps)_expe
ARC,,,,,,
MIDT,,,,,,
SEM Impressions,,,,,,
Direct vs Expedia,96.2%,330.0,390.0,,,
Kayak Placement,22.8%,1170.0,990.0,19.4%,920.0,870.0
Skyscanner Placement,18.7%,1450.0,1780.0,30.9%,-1330.0,-1310.0
US Outbound,8.0%,-90.0,-80.0,,,
Merchant Retail,17.4%,1060.0,920.0,,,
Merchant Total,26.7%,880.0,700.0,,,
deal_share,37.0%,90.0,-30.0,,,


'798K429K138K245K11K169K2K113K11K'

In [112]:

import config_table
import importlib

importlib.reload(config_table)
from docx import Document
from docx.oxml.ns import qn
from docx.oxml import OxmlElement
from docx.shared import Pt
from config_table import clear_document
from config_table import set_font
from config_table import create_word_table
from config_table import create_summary_table
from config_table import create_others_table
# from config_table import create_dau_table
from config_table import create_roi_table


# Data preparation and configurations
carrier_logos = [
    {'start_col': 1, 'end_col': 3, 'path': '../Screenshots/weeklynote/Retail.png', 'title': 'Retail'},
    {'start_col': 4, 'end_col': 6, 'path': '../Screenshots/weeklynote/Express.png', 'title': 'Express Deals'},
    {'start_col': 7, 'end_col': 9, 'path': '../Screenshots/weeklynote/Total.png', 'title': 'Total'}
]

business_logos = [
    {'start_col': 1, 'end_col': 3, 'path': '../Screenshots/weeklynote/Standalone.png', 'title': 'Standalone'},
    {'start_col': 4, 'end_col': 6, 'path': '../Screenshots/weeklynote/Package.png', 'title': 'Package'},
    {'start_col': 7, 'end_col': 9, 'path': '../Screenshots/weeklynote/Total.png', 'title': 'Total'}
]

channel_logos = [
    {'start_col': 1, 'end_col': 3, 'path': '../Screenshots/weeklynote/App.png', 'title': 'App'},
    {'start_col': 4, 'end_col': 6, 'path': '../Screenshots/weeklynote/MWeb Desktop.png', 'title': 'Web'},
    {'start_col': 7, 'end_col': 9, 'path': '../Screenshots/weeklynote/Total.png', 'title': 'Total'}
]

source_logos = [
    {'start_col': 1, 'end_col': 3, 'path': '../Screenshots/weeklynote/Published.png', 'title': 'Published'},
    {'start_col': 4, 'end_col': 6, 'path': '../Screenshots/weeklynote/Private.png', 'title': 'Private'},
    {'start_col': 7, 'end_col': 9, 'path': '../Screenshots/weeklynote/Total.png', 'title': 'Total'}
]

def customize_bullet_style(paragraph, font_size):
    """
    Customize bullet style in a paragraph using XML manipulation.
    """
    # Access the paragraph's properties
    pPr = paragraph._element.get_or_add_pPr()
    numPr = OxmlElement('w:numPr')
    ilvl = OxmlElement('w:ilvl')
    ilvl.set(qn('w:val'), '0')  # Set indentation level
    numId = OxmlElement('w:numId')
    numId.set(qn('w:val'), '1')  # Set numbering ID

    numPr.append(ilvl)
    numPr.append(numId)
    pPr.append(numPr)

    # Modify font properties
    for run in paragraph.runs:
        run.font.name = "Montserrat"
        run.font.size = font_size

# Create a Word Document
word_document = Document()

# Clear the document (if necessary)
clear_document(word_document)

# Add Title
word_document.add_paragraph()
word_document.add_paragraph(f'Summary')
set_font(word_document.paragraphs[-1], font_name="Montserrat", font_size=Pt(18), bold=True)
word_document.add_paragraph('\n')

# Create Summary Table
create_summary_table(word_document, df_summary)

# Add General Notes
summary_notes = [
    'All Priceline tickets (includes B2B, Package, Express Deals, Phone Sales)',
    'Normalized tickets are counted the same as tickets, except split tickets count as 1 instead of 2',
    'Refunds are assigned to refund date',
    'Revenue is contribution w/fee + GDS incentives.  Does not include package or phone sales.',
    'Daily Active Users: engaged customers in GA4',
    'TSA market share: travel date; numerator counts all slices (OW is 1 slice; RT is 2 slices) where the first segment is either US domestic or US outbound (Priceline-only); denominator includes all people passing through TSA screening machines'
]
for note in summary_notes:
    word_document.add_paragraph(note, style='List Bullet')
    set_font(word_document.paragraphs[-1], font_name="Montserrat", font_size=Pt(7), bold=False)
    customize_bullet_style(word_document.paragraphs[-1], font_size=Pt(7))

# Add Tables for Business, Carrier, Channel, and Source
tables = [
    (df_business, "Business", business_logos, [
        'All Priceline tickets (includes Express Deals and Phone Sales, not normalized)'
    ]),
    (df_carrier, "Carrier", carrier_logos, [
        'Priceline (including phone sales), B2C, Standalone (not normalized)'
    ]),
    (df_channel, "Channel", channel_logos, [
        'Priceline (including phone sales), B2C, Standalone (includes Express Deals, not normalized)'
    ]),
    (df_source, "Source", source_logos, [
        'Priceline (including phone sales), B2C, Standalone (includes Express Deals, not normalized)',
        'Indirect Connect = GDS + Aggregators + Consolidators + NDC-X'
    ])
]
for df, title, logos, notes in tables:
    if df is None:
        continue

   # Add specific custom notes for each table
    if title == "Business":
        custom_note = "Total Business - Detail"
    elif title == "Carrier":
        custom_note = "Priceline B2C Standalone - Detail"
    else:
        custom_note = " "

    # Add the custom note before the table
    word_document.add_paragraph(custom_note)
    set_font(word_document.paragraphs[-1], font_name="Montserrat", font_size=Pt(18), bold=True)
    word_document.add_paragraph()

    create_word_table(df, title, logos, word_document)

    # word_document.add_paragraph()

    for note in notes:
        word_document.add_paragraph(note, style='List Bullet')
        # word_document.add_paragraph('\n')
        set_font(word_document.paragraphs[-1], font_name="Montserrat", font_size=Pt(7), bold=False)

word_document.add_paragraph('\n')
word_document.add_paragraph('Other Metrics')
set_font(word_document.paragraphs[-1], font_name="Montserrat", font_size=Pt(18), bold=True)
create_others_table(word_document, df_others)

others_notes = [
    'ARC market share: OTA only, US POS, does not include NK/F9/SY',
    'SEM impression share: percentage of eligible impressions we show at the absolute top of the ads',
    'Direct parity vs Expedia: US origins, win+tie rate',
    'Kayak placement: US origins, availability in the top 4 positions',
    'Skyscanner placement: US origins, availability in the top 4 positions',
    'US outbound share: ticket share for US origins, international destinations',
    'Merchant share: ticket share where Priceline is merchant of record',
    'Deal share: share of tickets sold as a deal',
    'Conversion: GA4, converted customers / engaged customers',
    'Bookability: acceptance rate, includes multiple attempts by same user'

]
for note in others_notes:
    word_document.add_paragraph(note, style='List Bullet')
    set_font(word_document.paragraphs[-1], font_name="Montserrat", font_size=Pt(7), bold=False)
word_document.add_paragraph('\n')

word_document.add_paragraph(f'ROI')
set_font(word_document.paragraphs[-1], font_name="Montserrat", font_size=Pt(18), bold=True)

# Add ROI Section
# roi_notes = [
#     'ROI = Contribution / Cost',
#     'SEM Brand includes all products, weighted by ratio of flight orders',
#     'Meta includes Kayak/Momondo, includes Kayak credit',
#     'Last week'
# ]
create_roi_table(word_document, df_roi_v)

for note in notes:
        word_document.add_paragraph(note, style='List Bullet')
        set_font(word_document.paragraphs[-1], font_name="Montserrat", font_size=Pt(7), bold=False)
word_document.add_paragraph('\n')


# Save the Document
output_filename = os.path.join('../output/', f'Flight Performance Week Ending {end_date}.docx')
word_document.save(output_filename)
print(f"Word document '{output_filename}' has been created successfully.")

# Save PDF to Shared Drive
share_drive_path = '../../../Flight Weekly Note Output/'
word_document.save(share_drive_path + f'Flight Performance Week Ending {end_date}.pdf')

print(f"Word document saved to shared drive at '{share_drive_path}' has been created successfully.")


Word document '../output/Flight Performance Week Ending 2025-09-06.docx' has been created successfully.
Word document saved to shared drive at '../../../Flight Weekly Note Output/' has been created successfully.
